In [1]:
import napari
import numpy as np
from pathlib import Path
from skimage import io
import os
from qtpy.QtWidgets import QWidget, QVBoxLayout, QHBoxLayout, QComboBox, QPushButton, QLabel
from qtpy.QtCore import Qt
from scipy import ndimage
from qtpy.QtWidgets import QFileDialog

# 画像が含まれたディレクトリのパスを指定
image_dir = "/Users/sakanotakumi/Documents/000_Home/Work/Research/02_Experiments_data/E343/data/imagesTs-1.tif"
labels_dir = "/Users/sakanotakumi/Documents/000_Home/Work/Research/02_Experiments_data/E343/output/others/"

dendrite_dir = "/Users/sakanotakumi/Documents/000_Home/Work/Research/02_Experiments_data/E343/output/dendrite"
axon_dir = "/Users/sakanotakumi/Documents/000_Home/Work/Research/02_Experiments_data/E343/output/axon"

# サポートされる画像形式
supported_formats = ('.png', '.jpg', '.jpeg', '.tiff', '.tif', '.bmp')

def load_zstack_tiff(file_path):
    """
    z-stackされたTIFF画像を読み込む関数
    """
    try:
        image = io.imread(file_path)
        print(f"Loaded z-stack TIFF: {Path(file_path).name}, shape: {image.shape}")
        return image
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def load_images_from_directory(directory_path, supported_formats):
    """
    指定されたディレクトリから画像を読み込む関数
    """
    directory = Path(directory_path)
    images = []
    names = []
    
    if directory.exists():
        for image_file in sorted(directory.iterdir()):
            if image_file.suffix.lower() in supported_formats:
                image = io.imread(image_file)
                images.append(image)
                names.append(image_file.stem)
                print(f"Loaded: {image_file.name}")
    else:
        print(f"Directory {directory_path} does not exist")

    return np.array(images), names

# 画像ファイルを読み込み
images = load_zstack_tiff(image_dir)
labels, _ = load_images_from_directory(labels_dir, supported_formats)
dendrite = load_images_from_directory(dendrite_dir, supported_formats)[0]
axon = load_images_from_directory(axon_dir, supported_formats)[0]

# z-stack画像として結合
images_z_stack = np.stack(images, axis=0)
labels_z_stack = np.stack(labels, axis=0)

dendrite_z_stack = np.stack(dendrite, axis=0)
axon_z_stack = np.stack(axon, axis=0)

# napariビューアーを作成
viewer = napari.Viewer()

# napariにz-stack画像として追加
viewer.add_image(images_z_stack, name="Images")
viewer.add_labels(labels_z_stack, name="Labels")
viewer.add_labels(dendrite_z_stack, name="Dendrite")
viewer.add_labels(axon_z_stack, name="Axon")

class LabelTransferWidget(QWidget):
    def __init__(self, viewer):
        super().__init__()
        self.viewer = viewer
        self.init_ui()
    
    def init_ui(self):
        layout = QVBoxLayout()
        layout.setSpacing(5)  # スペーシングを小さく
        
        # From layer selection
        from_layout = QHBoxLayout()
        from_layout.addWidget(QLabel("From:"))
        self.from_combo = QComboBox()
        self.from_combo.setMinimumWidth(100)  # 最小幅を設定
        from_layout.addWidget(self.from_combo)
        layout.addLayout(from_layout)
        
        # To layer selection
        to_layout = QHBoxLayout()
        to_layout.addWidget(QLabel("To:"))
        self.to_combo = QComboBox()
        self.to_combo.setMinimumWidth(100)
        to_layout.addWidget(self.to_combo)
        layout.addLayout(to_layout)
        
        # Point layer selection
        point_layout = QHBoxLayout()
        point_layout.addWidget(QLabel("Points:"))
        self.point_combo = QComboBox()
        self.point_combo.setMinimumWidth(100)
        point_layout.addWidget(self.point_combo)
        layout.addLayout(point_layout)
        
        # Buttons
        self.update_button = QPushButton("Update")
        self.update_button.clicked.connect(self.update_layer_lists)
        layout.addWidget(self.update_button)
        
        self.transfer_button = QPushButton("Cut & Paste")
        self.transfer_button.clicked.connect(self.transfer_labels)
        layout.addWidget(self.transfer_button)
        
        # ウィジェット全体のサイズを制限
        self.setMaximumWidth(200)
        layout.setContentsMargins(5, 5, 5, 5)
        
        self.setLayout(layout)
        self.update_layer_lists()
        
        # Connect to layer events
        self.viewer.layers.events.inserted.connect(self.update_layer_lists)
        self.viewer.layers.events.removed.connect(self.update_layer_lists)
    
    def update_layer_lists(self):
        self.from_combo.clear()
        self.to_combo.clear()
        self.point_combo.clear()
        
        for layer in self.viewer.layers:
            if hasattr(layer, 'data') and 'Labels' in str(type(layer)):
                self.from_combo.addItem(layer.name)
                self.to_combo.addItem(layer.name)
            elif hasattr(layer, 'data') and 'Points' in str(type(layer)):
                self.point_combo.addItem(layer.name)
    
    def transfer_labels(self):
        from_layer_name = self.from_combo.currentText()
        to_layer_name = self.to_combo.currentText()
        point_layer_name = self.point_combo.currentText()
        
        if not all([from_layer_name, to_layer_name, point_layer_name]):
            print("Please select all required layers")
            return
        
        from_layer = None
        to_layer = None
        point_layer = None
        
        for layer in self.viewer.layers:
            if layer.name == from_layer_name:
                from_layer = layer
            elif layer.name == to_layer_name:
                to_layer = layer
            elif layer.name == point_layer_name:
                point_layer = layer
        
        if not all([from_layer, to_layer, point_layer]):
            print("Could not find selected layers")
            return
        
        points = point_layer.data
        if len(points) == 0:
            print("No points found in point layer")
            return
        
        for point in points:
            point_indices = tuple(int(coord) for coord in point)
            
            if any(idx < 0 or idx >= dim for idx, dim in zip(point_indices, from_layer.data.shape)):
                continue
            
            label_value = from_layer.data[point_indices]
            
            if label_value == 0:
                continue
            
            label_mask = (from_layer.data == label_value)
            to_layer.data[label_mask] = label_value
            from_layer.data[label_mask] = 0
        
        point_layer.data = []
        from_layer.refresh()
        to_layer.refresh()
        point_layer.refresh()
        print(f"Labels transferred from '{from_layer_name}' to '{to_layer_name}' and points cleared")

class LabelSaveWidget(QWidget):
    def __init__(self, viewer):
        super().__init__()
        self.viewer = viewer
        self.save_directory = Path("saved_labels_png")
        self.init_ui()
    
    def init_ui(self):
        layout = QVBoxLayout()
        layout.setSpacing(5)
        
        # Layer selection
        layer_layout = QHBoxLayout()
        layer_layout.addWidget(QLabel("Layer:"))
        self.save_combo = QComboBox()
        self.save_combo.setMinimumWidth(100)
        layer_layout.addWidget(self.save_combo)
        layout.addLayout(layer_layout)
        
        # Directory selection - 短縮表示
        dir_layout = QVBoxLayout()
        dir_layout.addWidget(QLabel("Save Directory:"))
        self.dir_label = QLabel("saved_labels_png")
        self.dir_label.setStyleSheet("border: 1px solid gray; padding: 2px; font-size: 10px;")
        self.dir_label.setWordWrap(True)  # テキストの折り返しを有効に
        self.dir_label.setMaximumHeight(40)  # 高さを制限
        dir_layout.addWidget(self.dir_label)
        
        self.browse_button = QPushButton("Browse")
        self.browse_button.clicked.connect(self.browse_directory)
        dir_layout.addWidget(self.browse_button)
        layout.addLayout(dir_layout)
        
        # Buttons
        self.update_save_button = QPushButton("Update")
        self.update_save_button.clicked.connect(self.update_layer_list)
        layout.addWidget(self.update_save_button)
        
        self.save_button = QPushButton("Save PNG")
        self.save_button.clicked.connect(self.save_labels_as_png)
        layout.addWidget(self.save_button)
        
        # ウィジェット全体のサイズを制限
        self.setMaximumWidth(200)
        layout.setContentsMargins(5, 5, 5, 5)
        
        self.setLayout(layout)
        self.update_layer_list()
        
        self.viewer.layers.events.inserted.connect(self.update_layer_list)
        self.viewer.layers.events.removed.connect(self.update_layer_list)
    
    def browse_directory(self):
        directory = QFileDialog.getExistingDirectory(
            self, 
            "Select Directory to Save Labels",
            str(self.save_directory.parent)
        )
        if directory:
            self.save_directory = Path(directory)
            # パスを短縮して表示
            display_path = str(self.save_directory)
            if len(display_path) > 30:
                display_path = "..." + display_path[-27:]
            self.dir_label.setText(display_path)
    
    def update_layer_list(self):
        self.save_combo.clear()
        for layer in self.viewer.layers:
            if hasattr(layer, 'data') and 'Labels' in str(type(layer)):
                self.save_combo.addItem(layer.name)
    
    def save_labels_as_png(self):
        layer_name = self.save_combo.currentText()
        if not layer_name:
            print("Please select a layer to save")
            return
        
        selected_layer = None
        for layer in self.viewer.layers:
            if layer.name == layer_name:
                selected_layer = layer
                break
        
        if selected_layer is None:
            print("Could not find selected layer")
            return
        
        self.save_directory.mkdir(exist_ok=True)
        label_data = selected_layer.data
        
        for z_idx in range(label_data.shape[0]):
            slice_data = label_data[z_idx]
            slice_data = slice_data.astype(np.uint16)
            filename = self.save_directory / f"{z_idx:04d}.png"
            io.imsave(filename, slice_data)
        
        print(f"Saved {label_data.shape[0]} PNG files to {self.save_directory}")

# ウィジェットを作成して追加
transfer_widget = LabelTransferWidget(viewer)
viewer.window.add_dock_widget(transfer_widget, name="Transfer", area="left")

save_widget = LabelSaveWidget(viewer)
viewer.window.add_dock_widget(save_widget, name="Save", area="left")

napari.run()


Loaded z-stack TIFF: imagesTs-1.tif, shape: (209, 700, 1321)
Loaded: 0000.png
Loaded: 0001.png
Loaded: 0002.png
Loaded: 0003.png
Loaded: 0004.png
Loaded: 0005.png
Loaded: 0006.png
Loaded: 0007.png
Loaded: 0008.png
Loaded: 0009.png
Loaded: 0010.png
Loaded: 0011.png
Loaded: 0012.png
Loaded: 0013.png
Loaded: 0014.png
Loaded: 0015.png
Loaded: 0016.png
Loaded: 0017.png
Loaded: 0018.png
Loaded: 0019.png
Loaded: 0020.png
Loaded: 0021.png
Loaded: 0022.png
Loaded: 0023.png
Loaded: 0024.png
Loaded: 0025.png
Loaded: 0026.png
Loaded: 0027.png
Loaded: 0028.png
Loaded: 0029.png
Loaded: 0030.png
Loaded: 0031.png
Loaded: 0032.png
Loaded: 0033.png
Loaded: 0034.png
Loaded: 0035.png
Loaded: 0036.png
Loaded: 0037.png
Loaded: 0038.png
Loaded: 0039.png
Loaded: 0040.png
Loaded: 0041.png
Loaded: 0042.png
Loaded: 0043.png
Loaded: 0044.png
Loaded: 0045.png
Loaded: 0046.png
Loaded: 0047.png
Loaded: 0048.png
Loaded: 0049.png
Loaded: 0050.png
Loaded: 0051.png
Loaded: 0052.png
Loaded: 0053.png
Loaded: 0054.png
Load

In [ ]:
temp = viewer.layers['axon'].data

2025-08-15 21:52:36.669 python[82554:20567070] error messaging the mach port for IMKCFRunLoopWakeUpReliable
Traceback (most recent call last):
  File "/var/folders/9g/_9g9gbr93_l2p9s0z7lzq7f00000gn/T/ipykernel_82554/1086947596.py", line 173, in transfer_labels
    to_layer.data[label_mask] = label_value
  File "/Users/sakanotakumi/Documents/000_Home/Work/Research/02_Experiments_data/E343/scripts/.venv/lib/python3.9/site-packages/dask/array/core.py", line 1926, in __setitem__
    dsk = setitem_array(out, self, key, value)
  File "/Users/sakanotakumi/Documents/000_Home/Work/Research/02_Experiments_data/E343/scripts/.venv/lib/python3.9/site-packages/dask/array/slicing.py", line 1646, in setitem_array
    indices, implied_shape, reverse, implied_shape_positions = parse_assignment_indices(
  File "/Users/sakanotakumi/Documents/000_Home/Work/Research/02_Experiments_data/E343/scripts/.venv/lib/python3.9/site-packages/dask/array/slicing.py", line 1335, in parse_assignment_indices
    raise Ind